In [5]:
import numpy as np
import os
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.applications import EfficientNetB0
from keras.models import Sequential
from keras import layers
import pickle
import tensorflow as tf

# Define a function to load and preprocess your data
def load_and_preprocess_data(data_dir):
    image_array = []
    label_array = []

    files = os.listdir(data_dir)

    for i in range(len(files)):
        file_sub = os.listdir(os.path.join(data_dir, files[i]))

        for k in tqdm(range(len(file_sub))):
            try:
                img = cv2.imread(os.path.join(data_dir, files[i], file_sub[k]))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (96, 96))
                image_array.append(img)
                label_array.append(i)
            except:
                pass

    image_array = np.array(image_array) / 255.0
    label_array = np.array(label_array)

    return image_array, label_array

# Load and preprocess your data
data_directory = "data"  # Update with your data directory
X, y = load_and_preprocess_data(data_directory)

# Split your data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.15)

# Define and compile your model
model = Sequential()
pretrained_model = EfficientNetB0(
    input_shape=(96, 96, 3),
    include_top=False,
    weights="imagenet"
)
model.add(pretrained_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1))

model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=["accuracy"]
)

# Train your model
EPOCHS = 35
BATCH_SIZE = 64

history = model.fit(
    X_train,
    Y_train,
    validation_data=(X_test, Y_test),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS
)

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model as a .tflite file
tflite_filename = "trained_model.tflite"
with open(tflite_filename, 'wb') as tflite_file:
    tflite_file.write(tflite_model)


100%|██████████| 50/50 [00:00<00:00, 684.96it/s]


Epoch 1/35
2/2 [==============================] - 20s 4s/step - loss: 1.4208 - accuracy: 0.6706 - val_loss: 1.3184 - val_accuracy: 0.6000
Epoch 2/35
2/2 [==============================] - 4s 2s/step - loss: 0.8288 - accuracy: 0.5882 - val_loss: 1.3167 - val_accuracy: 0.6000
Epoch 3/35
2/2 [==============================] - 3s 1s/step - loss: 0.3744 - accuracy: 0.9059 - val_loss: 0.9261 - val_accuracy: 0.6000
Epoch 4/35
2/2 [==============================] - 4s 2s/step - loss: 0.4681 - accuracy: 0.7647 - val_loss: 0.8200 - val_accuracy: 0.6000
Epoch 5/35
2/2 [==============================] - 4s 1s/step - loss: 0.3509 - accuracy: 0.9059 - val_loss: 0.7972 - val_accuracy: 0.6000
Epoch 6/35
2/2 [==============================] - 7s 2s/step - loss: 0.2420 - accuracy: 0.7529 - val_loss: 0.6976 - val_accuracy: 0.6000
Epoch 7/35
2/2 [==============================] - 4s 1s/step - loss: 0.3298 - accuracy: 0.8706 - val_loss: 0.6649 - val_accuracy: 0.6000
Epoch 8/35
2/2 [========================

INFO:tensorflow:Assets written to: C:\Users\rushi\AppData\Local\Temp\tmp6yd9zvps\assets


INFO:tensorflow:Assets written to: C:\Users\rushi\AppData\Local\Temp\tmp6yd9zvps\assets
